In [ ]:
import os
import getpass

import gretel_client as gretel
import pandas as pd

In [ ]:
DATA_SUBSET = "Video_Games_v1_00"

api_key = os.getenv("GRETEL_API_DEV_KEY") or getpass.getpass("Enter your Gretel API key: ")

gretel.configure_session(
    api_key=api_key,
    endpoint="https://api-dev.gretel.cloud",
    validate=True,
    clear=True,
)

project = gretel.projects.get_project(name=f"gretel-gpt-sentiment-swap-{DATA_SUBSET.replace('_', '-')}")

In [ ]:
project = gretel.projects.get_project(name=f"gretel-gpt-sentiment-swap-{DATA_SUBSET.replace('_', '-')}")

In [ ]:
# get model id of the last completed model associated with the the data subset
model= [m for m in project.search_models(model_name=DATA_SUBSET) if m.status == "completed"][-1]

record_handler = model.create_record_handler_obj(
    params={"maximum_text_length": 200, "temperature": 1.2}, 
    data_source=pd.read_csv(f"data/conditional_prompts_{DATA_SUBSET}.csv.gz")
)

record_handler.submit_cloud()
gretel.helpers.poll(record_handler)

df_generations = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df_generations.to_csv(f"model-generations/generations-{DATA_SUBSET}.csv.gz", index=False)

df_prompts = pd.read_csv(f"data/conditional_prompts_{DATA_SUBSET}.csv.gz")

In [ ]:
num_samples = 5

samples = df_prompts.sample(num_samples)
for idx, prompt in samples.itertuples():
    generation = df_generations.loc[idx, "text"]
    print(f"{prompt} \033[1;30;46m{generation}\033[0;0m\n-----\n")